In [2]:
import xgboost as xgb
import pandas as pd
import os 
import numpy as np
import sys
import gc
from time import time
from pathos.multiprocessing import ProcessingPool as Pool

In [3]:
def pred (model_path,data,out_path,out_name):
    model = xgb.Booster(model_file=model_path)
    out_data = pd.read_csv(data)
    
    pred_data = out_data.drop(columns=['lon','lat'],axis = 1)#
    data_dmatrix = xgb.DMatrix(pred_data)
    pred = model.predict(data=data_dmatrix)
    
    out_data["bird"] = pred
    out_data = out_data[["lon","lat","bird"]]
    out_data.to_csv(out_path+out_name+'.csv',index=False)

In [7]:
model_path = r'../models/'## the models trained in train_model.ipynb
basemap_path = r'../basemap/'## a basemap include the environment variables(table S2),the grid size is consistent with era5-land
out_path = r'../bird_distribution_map/csv/'##  Model prediction results in grid: (latitude and longitude) - migratory bird probability 
t1 = time()
os.chdir(model_path)
flist = os.listdir()
pool = Pool(4)
for i in flist:
    filetype = os.path.splitext(i)[1]
    modelname = os.path.splitext(i)[0]
    if filetype == '.json':
        #以下要遍历所有的basemap，依次做出预测，并输出为csv
        print(i)
        #creat floder
        os.mkdir(out_path+modelname)
        p_map = []#data
        p_mapname = []#out_name
        for bmap in os.listdir(basemap_path):
            filetype2 = os.path.splitext(bmap)[1]
            bmapname = os.path.splitext(bmap)[0]
            if filetype2 == '.csv':
                p_map.append(basemap_path+bmap)
                p_mapname.append(bmapname)              
        p_model = [model_path+i]*len(p_map)#model
        p_out_path = [out_path+modelname+'/']*len(p_map)#out_path
        #params = zip(p_model,p_map,p_out_path,p_mapname)
        pool.map(pred,p_model,p_map,p_out_path,p_mapname)
        print(time()-t1)
        t1 = time()
        
    #pool.terminate()
pool.close()
pool.join()

Gruiformes.json
19.085556745529175
Procellariiformes.json
16.05157732963562
Accipitriformes.json
20.091309785842896
Passeriformes.json
19.171802759170532
Ciconiiformes.json
20.089248180389404
Falconiformes.json
19.454153299331665
Charadriiformes.json
18.961719512939453
Pelecaniformes.json
18.55037808418274
Anseriformes.json
19.282253980636597
Suliformes.json
15.223488807678223


In [ ]:
## Convert the grid result to TIF

In [ ]:
proj = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'
trans = (-180,0.25,0,90,0,-0.25)
img_width, img_height=1440,721
order_list = ['Gruiformes', 'Procellariiformes', 'Accipitriformes', 'Passeriformes', 'Ciconiiformes', 'Falconiformes', 'Charadriiformes', 'Pelecaniformes', 'Anseriformes', 'Suliformes']

In [ ]:
infile = r'../bird_distribution_map/csv/'
outpath =  r'../bird_distribution_map/tif/'   

In [ ]:
def buil_tif(infile,outpath,outname):
        
    datatype = gdal.GDT_Float32
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(outpath+outname+".tif",img_width,img_height, 1, datatype)
    
    pred = pd.read_csv(infile,engine = 'c')
    pred.lon = pred.lon.apply(lambda x: x-360  if x>180 else x)
    pred = pred.sort_values(by=['lat','lon'],ascending=[False,True])
    bird = pred.bird.values
    outif = np.matrix(bird.reshape(img_height,img_width))
   
    dataset.SetGeoTransform(trans)  # 写入仿射变换参数
    dataset.SetProjection(proj)  # 写入仿射变换参数
    dataset.GetRasterBand(1).WriteArray(outif)
    del dataset

In [ ]:
t1=time()
pool = Pool(4)

for order in order_list:
    print(order)
    os.chdir(infile+order)
    map_csv = []
    raster_out = []
    raster_name = []
    for bmap in os.listdir():
        filetype = os.path.splitext(bmap)[1]
        bmapname = os.path.splitext(bmap)[0]
        if filetype == '.csv':
            map_csv.append(infile+order+'/'+bmap)
            raster_out.append(outpath+order)
            raster_name.append(bmapname)
    pool.map(buil_tif,map_csv,raster_out,raster_name)
    #buil_tif(map_csv[0],raster_out[0],raster_name[0])
    print(time()-t1)
    t1 = time()
pool.close()
pool.join()
